In [1]:
from data_preparation import DataPreparator
from apply_clustering import ClusteringApplier

In [2]:
import pandas as pd
import numpy as np

In [3]:
import config

In [4]:
# Run this to make sure the "normal way" still works
VALIDATE_THE_OTHER_WAY = False

if VALIDATE_THE_OTHER_WAY:
    DataPreparator.prepare_data()
    optimal_number_of_clusters = ClusteringApplier.draw_gap_statistic_plot()
    ClusteringApplier.calculate_observable_patterns(_n_clusters=optimal_number_of_clusters)
    ClusteringApplier.calculate_explainable_distances()

In [5]:
df_observable = pd.read_excel(config.INPUT_FILE_OBSERVABLE_FEATURES) #'../data/biological_set_raw_data.xlsx')
df_explainable = pd.read_excel(config.INPUT_FILE_EXPLAINING_FEATURES)#'../data/biological_set_explaining_features.xlsx')

In [6]:
df_observable.Country = df_observable.Country.transform(lambda x: x.strip())

In [7]:
#observed_features = ['perc_lying', 'perc_lhd', 'num_lying', 'num_lhd']
#explaining_features = ['Age', 'Sex', 'Zoo', 'Stable', 'Genus_ID', 'Family_ID','Order_ID', 'SH', 'Weight', 'Habitat']
#index_name = 'Individual'

observed_features = list(config.OBSERVABLE_FEATURE_NAMES.keys())
explaining_features = list(config.EXPLAINING_FEATURE_NAMES.keys())
index_name = config.GROUP_NAME

In [8]:
preparation_result = DataPreparator.prepare_data(
    use_config=False,
    df_observable=df_observable,
    df_explainable=df_explainable,
    distortion_mean=0.,
    distortion_std=0.001,
    observed_features=observed_features,
    explaining_features=explaining_features,
    nn_imputation_k=10,
    group_name=index_name
)

In [9]:
optimal_number_of_clusters = ClusteringApplier.draw_gap_statistic_plot(
    use_config=False, df_observable=preparation_result['observable_df'],
    observed_features=observed_features,
    gap_statistic_cluster_range=10,
    plot=False
)

In [10]:
optimal_number_of_clusters

{'n_clusters': 9,
 'knee': np.int64(2),
 'elbow': np.int64(2),
 'clusterer': OptimalK(n_jobs=16, parallel_backend="joblib")}

In [11]:
...

Ellipsis

In [13]:
observable_patterns_result = ClusteringApplier.calculate_observable_patterns(
    use_config=False,
    df_observable_data=preparation_result['observable_df'],
    _n_clusters=optimal_number_of_clusters['n_clusters'],
    observable_name='observable_name',
    observable_feature_names=observed_features,
    plot_title='Name of the plot',
    number_observable_patterns='auto',
    distance_measure_fingerprint='jensenshannon',
    group_name=index_name, 
    plot=False
)

In [16]:
calculate_explainable_distances_result = ClusteringApplier.calculate_explainable_distances(
    use_config=False,
    df_explainable=preparation_result['explainable_df'].copy(),
    df_observable_distances=observable_patterns_result['pw_norm_dist'].copy(),
    explaining_features=explaining_features,
    method='exact',
    distance_measure='correlation',
    sparsity_parameter=0.02,
    max_num_threads=12,
    group_name=index_name,
    plot=False
)

  0%|          | 0/65535 [00:00<?, ?it/s]

In [17]:
heuristic_calculate_explainable_distances_result = ClusteringApplier.calculate_explainable_distances(
    use_config=False,
    df_explainable=preparation_result['explainable_df'].copy(),
    df_observable_distances=observable_patterns_result['pw_norm_dist'].copy(),
    explaining_features=explaining_features.copy(),
    method='heuristic',
    distance_measure='correlation',
    sparsity_parameter=0.02,
    max_num_threads=12,
    group_name=index_name,
    heuristics_N=2,
    plot=False
)
heuristic_calculate_explainable_distances_result['optimal_feature_set']

  0%|          | 0/136 [00:00<?, ?it/s]

Started with ['HDI', 'LPI']
Refuted 'GI'


['HDI', 'LPI']

In [14]:
calculate_explainable_distances_result['overview_df']

NameError: name 'calculate_explainable_distances_result' is not defined